# NLP Classification

In this assignment we look at several ways of classifying texts:
- Naive Bayes
- Logistic Regression
- Multinomial Regression

We also look at binary label classification problems (e.g., sentiment analysis) and multinomial classification problems (e.g., topic analysis).

We will use two datasets:
- [IMDb movie review sentiment](http://ai.stanford.edu/~amaas/data/sentiment/)
- [AG News topics](https://huggingface.co/datasets/ag_news)

**Tips:**
- Read all the code. We don't ask you to write the training loops, or evaluation loops, but it is often instructive to see how the models are trained and evaluated.
- If you have a model that is learning (loss is decreasing), but you want to increase accuracy, try using ``nn.Dropout`` layers just before the final linear layer to force the model to handle missing or unfamiliar data.

In [1]:
# start time - notebook execution
import time
start_nb = time.time()

# Set up

Import packages.

In [2]:
import nltk
import numpy as np
import os
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim

# Initialize the Autograder

In [3]:
# import the autograder tests
import hw2_tests as ag

# Functions for cleaning up raw texts and tokenizing the corpus

We perform text preprocessing that includes: removing HTML tags, making text lower case, stemming, and disposing of stopwords.
In the end, we will split the entire dataset into training, validation and test sets.

In [4]:
# Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= [ps.stem(word) for word in text]
    return text

In [5]:
stopwords_english = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
print(stopwords_english)

#removing the stopwords
def remove_stopwords(text, stopword_list):
    tokens = [token.strip() for token in text]
    filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    return filtered_tokens

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [6]:
def tokenize_and_clean(line, stem_and_remove_stop_words = True):

    line = re.sub(r"<.*?>", "", line).strip() # remove all HTML tags
    line = re.sub(r'[^a-zA-Z0-9]', ' ', line) # remove punc
    line = line.lower().split()  # lower case
    if stem_and_remove_stop_words:
        line = remove_stopwords(line, stopwords_english)
        line = simple_stemmer(line)

    return line

# Download and unpack the sentiment data



We are using IMDb Dataset for binary sentiment classification that provides a set of 25K highly polar reviews for training, and 25K for testing
(each set contains an equal number of positive and negative examples).

Dataset folder structure is as follows:

dataset/ \
├── test/ \
│     ├── pos/ \
│     ├── neg/ \
├── train/ \
      ├── pos/ \
      └── neg/

In [7]:
# check if dataset is downloaded
# if not os.path.isfile('aclImdb_v1.tar'):
#     print("Downloading dataset...")
#     !wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
#     !gunzip aclImdb_v1.tar.gz
#     !tar -xvf aclImdb_v1.tar

Load in the text from the folders.

In [8]:
def load_text_from_folders(path, file_list, dataset, samples = 2000, stem_and_remove_stop_words = True):
    """Read set of files from given directory and save returned lines to list.

    Parameters
    ----------
    path : str
        Absolute or relative path to given file (or set of files).
    file_list: list
        List of files names to read.
    dataset: list
        List that stores read lines.
    samples: int
        Number of samples in the output
    """
    for i, file in enumerate(file_list):
        if i >= samples:
            break
        with open(os.path.join(path, file), 'r', encoding='utf8') as text:
            contents = text.read()
            contents_tokenized = tokenize_and_clean(contents, stem_and_remove_stop_words=stem_and_remove_stop_words)
            dataset.append(contents_tokenized)

# Creating training and test sets

This creates four arrays:


*   ```train_pos``` -- instances in the training set with positive sentiment labels
*   ```train_neg``` -- instances in the training set with negative sentiment labels
*   ```test_pos``` -- instances in the testing set with positive sentiment labels
*   ```test_neg``` -- instances in the testing set with negative sentiment labels





In [9]:
# Path to dataset location
path = 'aclImdb/'

# Create lists that will contain read lines
train_pos, train_neg, test_pos, test_neg = [], [], [], []

# Create a dictionary of paths and lists that store lines (key: value = path: list)
sets_dict = {'train/pos/': train_pos, 'train/neg/': train_neg,
             'test/pos/': test_pos, 'test/neg/': test_neg}

# Load the data
for dataset in sets_dict:
  file_list = [f for f in sorted(os.listdir(os.path.join(path, dataset))) if f.endswith('.txt')]
  load_text_from_folders(os.path.join(path, dataset), file_list, sets_dict[dataset])

Convert into Pandas dataframes. Pandas is a virtual spreadsheet with a programmatic API. A ```DataFrame``` is a spreadsheet. We will make a spreadsheet of training data and one for testing data and one with everything together.

In [10]:
# Concatenate training and testing examples into one dataset
TRAIN = pd.concat([pd.DataFrame({'review': train_pos, 'label':1}),
                     pd.DataFrame({'review': train_neg, 'label':0})],
                     axis=0, ignore_index=True)

TEST = pd.concat([pd.DataFrame({'review': test_pos, 'label':1}),
                    pd.DataFrame({'review': test_neg, 'label':0})],
                    axis=0, ignore_index=True)

ALL = pd.concat([TRAIN, TEST])

Look at the data.

This is a summary of the data. We see that the data is balanced between labels

In [11]:
TRAIN.label.value_counts()

label
1    2000
0    2000
Name: count, dtype: int64

This is the first few rows of the training set:

In [12]:
TRAIN.head()

,review,label
0,"[bromwel, high, cartoon, comedi, ran, time, pr...",1
1,"[homeless, houseless, georg, carlin, state, is...",1
2,"[brilliant, act, lesley, ann, warren, best, dr...",1
3,"[easili, underr, film, inn, brook, cannon, sur...",1
4,"[typic, mel, brook, film, much, less, slapstic...",1


# Creating a vocabulary file

Next, we have to build a vocabulary. This is effectively a look-up table where every unique word in your data set has a corresponding index (an integer).
We do this as our machine learning model cannot operate on strings, but only numbers. Each index is used to construct a one-hot vector for each word.

In [13]:
class Vocab:
    def __init__(self, name):
        self.name = name
        self._word2index = {}
        self._word2count = {}
        self._index2word = {}
        self._n_words = 0

    def get_words(self):
      return list(self._word2count.keys())

    def num_words(self):
      return self._n_words

    def word2index(self, word):
      return self._word2index[word]

    def index2word(self, word):
      return self._index2word[word]

    def word2count(self, word):
      return self._word2count[word]

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self._word2index:
            self._word2index[word] = self._n_words
            self._word2count[word] = 1
            self._index2word[self._n_words] = word
            self._n_words += 1
        else:
            self._word2count[word] += 1

Make a vocab object.

In [14]:
VOCAB = Vocab("imdb")
VOCAB_SIZE = 1000
NUM_LABELS = 2

Load the first ```n``` frequent words in the vocabulary. Do this by sorting by frequency and then truncating.

In [15]:
# Get word frequency counts
word_freq_dict = {}   # key = word, value = frequency
for review in ALL['review']:
  for word in review:
    if word in word_freq_dict:
      word_freq_dict[word] += 1
    else:
      word_freq_dict[word] = 1

# Get a list of (word, freq) tuples sorted by frequency
kv_list = []  # list of word-freq tuples so can sort
for (k,v) in word_freq_dict.items():
  kv_list.append((k,v))
sorted_kv_list = sorted(kv_list, key=lambda x: x[1], reverse=True)

# Load top n words in to vocab object
for word, freq in sorted_kv_list[:VOCAB_SIZE]:
  VOCAB.add_word(word)

# Naive Bayes
Naive Bayes Algorithm is based on the Bayes Rule which describes the probability of an event,
based on prior knowledge of conditions that might be related to the event.

According to Bayes theorem:


```Posterior = likelihood * proposition/evidence```

or

```P(A|B) = P(B|A) * P(A)/P(B)```


Using word presence as features, create an array of features for each review. Each review will thus be an array of size ```len(vocab)``` where each index in the array is a token number and the value in that position is whether the token is present in the review. There will be ```num_rows``` arrays, making a ```num_rows x len(vocab)``` 2D array.

This function creates a bag of words. It returns a vector where each element is a count of the words in the sentence corresponding to the word index.

In [16]:
def make_bow(sentence):
    vec = torch.zeros(VOCAB_SIZE, dtype=torch.float64)
    for word in sentence:
        if word not in VOCAB.get_words():
            continue
        vec[VOCAB.word2index(word)] += 1
    return vec.view(1, -1)

Prepare data ```X_TRAIN``` is a 2D array of size ```num_reviews x vocab_size``` that contains training data. Each row will be a bag of words, except each index contains a 1 or 0 based on word presence in the example. Each row is a vector of features $\phi_1 ... \phi_{|V|}$ assumed to be independent, where $|V|$ is size of the vocabulary. We don't need to know what the features are, only whether they are present in each example in the training set.

```X_TEST``` is the same as above but containing testing data.



In [17]:
# Vectorize text reviews to numbers
# Make empty vectors
X_TRAIN = np.zeros((len(TRAIN), VOCAB_SIZE))
X_TEST = np.zeros((len(TEST), VOCAB_SIZE))

# Load in frequency counts
for i, row in TRAIN.iterrows():
    X_TRAIN[i] = np.array(make_bow(row['review'])) > 0 # The > 0 converts to presence instead of counts

for i, row in TEST.iterrows():
    X_TEST[i] = np.array(make_bow(row['review'])) > 0 # The > 0 converts to presence instead of counts

# The labels
Y_TRAIN = np.array(TRAIN['label'])
Y_TEST = np.array(TEST['label'])

What you want to do is to compute probabilities over the training data and then apply those probabilities to the testing examples. Use the Bayes formula to compute $P_{\rm test}(L_{+}|\phi_{0:|V|})$ and $P_{\rm test}(L_{-}|\phi_{0:|V|})$ for each review. Classify examples based on whether one probability is higher than another. That is, $sign(P_{\rm test}(L_{+}|\phi_{0:|V|}) - P_{\rm test}(L_{-}|\phi_{0:|V|}))$ indicates a positive review when greater than 0 and a negative review when less than 0.

**Hint:** You do not need to implement any loops. Numpy indexing and slicing operations, along with built in functions like `.mean()`, `.sum()`, etc. will allow all operations to be performed on each row of the data in parallel.

Step 1: Compute the positive label condition:
$P(L_{+}|\phi_{0:|V|}) = P(\phi_{0:|V|}|L_{+})P(L_{+}) / P(\phi_{0:|V|})$

In [29]:
def grab_samples(xt, yt, pos):
  if pos:
    return xt[yt==1]
  else:
    return xt[yt==0]

def prob_pos_given_features(x_train, y_train):
  log_probs = np.array([0] * x_train.shape[1])
  ### BEGIN SOLUTION
  # so firt we grab all the positives, well grab all the negatives for the next function
  x_pos = grab_samples(x_train, y_train, True)

  # we now need the word counts for each and total_words
  # this jsut sums each of the words
  w_ct = np.sum(x_pos, axis=0) + 1
  # and now the total pos words for the denominator
  w_tot = np.sum(x_pos, axis=0) + 2

  # and now we just do the log of those guys for our return value
  log_probs = np.log(w_ct / w_tot)
  

  ### END SOLUTION
  return log_probs

Step 2: Compute the negative label condition:
$P(L_{-}|\phi_{0:|V|}) = P(\phi_{0:|V|}|L_{-})P(L_{-}) / P(\phi_{0:|V|})$

In [30]:
def prob_neg_given_features(x_train, y_train):
  log_probs = np.array([0] * x_train.shape[1])
  ### BEGIN SOLUTION
  # This is all gonna be the same as above except with negs
  # so firt we grab all the positives, well grab all the negatives for the next function
  x_neg = grab_samples(x_train, y_train, False)

  # we now need the word counts for each and total_words
  # this jsut sums each of the words
  w_ct = np.sum(x_neg, axis=0) + 1
  # and now the total pos words for the denominator
  w_tot = np.sum(x_neg, axis=0) + 2

  # and now we just do the log of those guys for our return value
  log_probs = np.log(w_ct / w_tot)
  
  ### END SOLUTION
  return log_probs

In [32]:
pos_probs = prob_pos_given_features(X_TRAIN, Y_TRAIN)
neg_probs = prob_neg_given_features(X_TRAIN, Y_TRAIN)
print(pos_probs)
print(neg_probs)

[-0.0008507  -0.00085653 -0.00085874 -0.00105541 -0.00117302 -0.00127146
 -0.00145455 -0.00152323 -0.001386   -0.00142755 -0.00126823 -0.00141343
 -0.00180018 -0.00177462 -0.0020141  -0.00180343 -0.00142959 -0.00203046
 -0.00184332 -0.00399202 -0.00203459 -0.0019861  -0.00207254 -0.0022805
 -0.00195886 -0.00148258 -0.00172563 -0.00198216 -0.00187441 -0.00179051
 -0.00165975 -0.00231214 -0.00215285 -0.0020555  -0.00218579 -0.00247832
 -0.00245399 -0.00211193 -0.00224972 -0.00270636 -0.00322061 -0.00276625
 -0.00228572 -0.00240674 -0.00209864 -0.00264901 -0.00202225 -0.00224972
 -0.00230681 -0.00239521 -0.00247219 -0.0025413  -0.00243605 -0.00347826
 -0.00241838 -0.0032949  -0.00302572 -0.00260756 -0.00238949 -0.00280505
 -0.00249066 -0.00332779 -0.00275862 -0.00407333 -0.00336135 -0.00383878
 -0.00392928 -0.00344234 -0.00351494 -0.00298063 -0.00332779 -0.0032
 -0.00508907 -0.00410678 -0.00373832 -0.00415801 -0.00270636 -0.00386848
 -0.00317965 -0.00583092 -0.0038835  -0.00312013 -0.0035

Step 3: Make a label prediction. Subtract (in log scale) the positive from the negative. If the result is greater than zero then it is a prediction of `+` label. If the result is less thn zero then we make a prediction of `-` label.

In [33]:
def naive_bayes(x, pos_probs, neg_probs):
  label = 0
  ### BEGIN SOLUTION
  # so we need to get the sum of all the pos and all the neg for sample x and 
  # just see which one is greater
  # and that will be the label we choose

  pos_total_logprob = np.sum(x * pos_probs)
  neg_total_logprob = np.sum(x * neg_probs)
  
  # this should do it
  if pos_total_logprob > neg_total_logprob:
    label = 1
  else:
    label = 0

  ### END SOLUTION
  return label

# Naive Bayes Test (20 Points)

In [34]:
# student check - accuracies >= 78% will receive full credit (no credit for less than 78%)
ag.test_naive_bayes(X_TRAIN, Y_TRAIN, X_TEST, Y_TEST)

Accuracy:  0.8155
Test A: 20/20


# Logistic Regression - Part 1

We will be using a neural network to perform logistic regression. We will use word counts as the input feature vector.


Reload the data, but use word counts instead of word presence.

In [35]:
# Randomize the data
TRAIN = TRAIN.sample(frac=1).reset_index(drop=True)
TEST = TEST.sample(frac=1).reset_index(drop=True)

# Vectorize text reviews to numbers
X_TRAIN = np.zeros((len(TRAIN), VOCAB_SIZE))
X_TEST = np.zeros((len(TEST), VOCAB_SIZE))

for i, row in TRAIN.iterrows():
  X_TRAIN[i] = np.array(make_bow(row['review']))

for i, row in TEST.iterrows():
  X_TEST[i] = np.array(make_bow(row['review']))

Y_TRAIN = np.array(TRAIN['label'])
Y_TEST = np.array(TEST['label'])

Make a logistic classifier torch neural network.

Complete the constructor and forward function. The net will take an arbitrary number of outputs, but for binary logistic regression, only one is needed because the single output neuron can take a value that is between 0 and 1, with 0 meaning negative sentiment and 1 meaning positive sentiment. There should only be as many parameters as ```num_features x (num_labels-1)``` in binary logistic regression and ```num_features x num_labels``` for multinomial logistic regression.

The input will be a one-hot vector of size `vocab_size`.

In [37]:
# Defining neural network structure
class BoWClassifier(nn.Module):  # inheriting from nn.Module!

  def __init__(self, num_labels, vocab_size):
    super(BoWClassifier, self).__init__()

    # BEGIN SOLUTION
    # so lets make the linear layer
    # these parameter sizes should be good as mentioned in the comments above
    # binary classification so num_lables should be 2? maybe 1? might have to check later
    self.lin1 = nn.Linear(vocab_size, num_labels)
    # END SOLUTION

  def forward(self, bow_vec):
    # BEGIN SOLUTION
    #  I think this is right, seems simple but its only one layer so all we need 
    # is activation function and we said wed use sigmoid in the notes
    out = torch.sigmoid(self.lin1(bow_vec))
    # END SOLUTION
    return out

In [38]:
# Initialize the model
# Use one label because the head can signify a 1 or 0 because of the sigmoid.
bow_nn_model = BoWClassifier(NUM_LABELS-1, VOCAB_SIZE)

This function should return two tensors. The first, containing training data, shoud be of size ```batch_size x vocab_size``` for the ```i```th batch. The second should be a list of labels of size ```batch_size```. Both tensors should be of type ```dtype=torch.float```.

In [41]:
def get_batch(i, batch_size, x_data, y_data):
  # Make some empty tensors
  x = None
  y = None
  ### BEGIN SOLUTION
  # copy nad paste this guy from HW1
  idx_st = i * batch_size
  idx_ed = idx_st + batch_size

  x_ind_tuples = x_data[idx_st: idx_ed]
  y_ind_tuples = y_data[idx_st: idx_ed]

  # separate the x from the y and convert to torch
  x_feats = torch.tensor(x_ind_tuples, dtype=torch.float32)
  y_feats = torch.tensor(y_ind_tuples, dtype=torch.float32)

  # had an error, needed to keep these return vals
  x = x_feats
  y = y_feats
  ### END SOLUTION
  return x, y

# Logistic Regression - Part 1 Test (20 Points)

In [42]:
# student check
ag.test_batch_output_shape(get_batch, X_TRAIN, Y_TRAIN, VOCAB_SIZE)

Output shape looks good!
Test B: 5/5


In [43]:
# student check - your model must have the expected number of layers to receive full credit, no credit otherwise
ag.check_bow_architecture(bow_nn_model)

Model has the expected number of layers.
Test C: 5/5
First layer is a Linear layer.
Test D: 5/5


In [44]:
# student check
ag.test_forward_pass_shape(X_TRAIN, Y_TRAIN, bow_nn_model)

Forward pass output shape looks good!
Test E: 5/5


# Logistic Regression - Part 2

Create a dataset as an array of (X_train, label).

Complete ```get_batch(i)``` and set ```batch_size``` and ```num_epochs```.

Training loop will call ```get_batch()``` with the iteration number and do everything else.


In [45]:
# Train the model
def train(model, train_data, test_data, epochs, batch_size):
  n_iter = len(train_data) // batch_size
  print(n_iter, 'batches per epoch')
  # Loss Function
  loss_function = nn.BCELoss()
  # Optimizer initlialization
  optimizer = optim.SGD(bow_nn_model.parameters(), lr=0.1)

  for epoch in range(epochs):
    # Make BOW vector for input features and target label
    for i in range(n_iter):
      x, y = get_batch(i, batch_size, train_data, test_data)

      # Step 3. Run the forward pass.
      y_hat = model(x)
      y_hat = y_hat.reshape(-1)

      # Step 4. Compute the loss, gradients, and update the parameters by
      loss = loss_function(y_hat,y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if (epoch+1)%10 == 0 and (i+1) == n_iter:
        print('epoch:', epoch+1,',loss =',loss.item(), ', training accuracy =',(torch.round(y_hat)==y).float().mean())
  return model

In [46]:
# It's ok to modify this cell.
BATCH_SIZE = 100
N_EPOCHS = 1000

In [47]:
try:
    bow_nn_model = train(bow_nn_model, X_TRAIN, Y_TRAIN, N_EPOCHS, BATCH_SIZE)
except:
    print("Training failed. Please check your code.")

40 batches per epoch
epoch: 10 ,loss = 0.2961275279521942 , training accuracy = tensor(0.9200)
epoch: 20 ,loss = 0.2470795065164566 , training accuracy = tensor(0.9300)
epoch: 30 ,loss = 0.22138631343841553 , training accuracy = tensor(0.9300)
epoch: 40 ,loss = 0.20494748651981354 , training accuracy = tensor(0.9300)
epoch: 50 ,loss = 0.19324444234371185 , training accuracy = tensor(0.9400)
epoch: 60 ,loss = 0.1843004673719406 , training accuracy = tensor(0.9400)
epoch: 70 ,loss = 0.17712117731571198 , training accuracy = tensor(0.9400)
epoch: 80 ,loss = 0.17115424573421478 , training accuracy = tensor(0.9500)
epoch: 90 ,loss = 0.16606755554676056 , training accuracy = tensor(0.9600)
epoch: 100 ,loss = 0.16164778172969818 , training accuracy = tensor(0.9600)
epoch: 110 ,loss = 0.15775053203105927 , training accuracy = tensor(0.9600)
epoch: 120 ,loss = 0.1542736291885376 , training accuracy = tensor(0.9600)
epoch: 130 ,loss = 0.15114210546016693 , training accuracy = tensor(0.9600)
epoc

# Logistic Regression - Part 2 Test (20 Points)

In [48]:
# student check - accuracies >= 78% will receive full credit (no credit for less than 78%)
ag.test_model_accuracy_lr(TEST, bow_nn_model)

              precision    recall  f1-score   support

           0       0.82      0.83      0.82      2000
           1       0.82      0.82      0.82      2000

    accuracy                           0.82      4000
   macro avg       0.82      0.82      0.82      4000
weighted avg       0.82      0.82      0.82      4000

Accuracy:  0.8
Test F: 20/20


# Multinomial Regression

Load data.

In [49]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\mccar\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [50]:
from datasets import load_dataset

Unlike earlier, we will use a pre-defined set of embeddings, called [GLoVe](https://nlp.stanford.edu/projects/glove/). GLoVe replaces every word with a 100-dimensional vector of floating point values. The advantage of this is that words with similar semantic meanings will have similar vectors. This is important because the vocabulary size of the corpus we will use is 400,000.

For the assigment, instead of getting a one-hot vector for each word, the neural network will get a `batch_size x num_words x 100` tensor containing floating point values.

Download the GLoVe embedding vectors.

In [51]:
import gensim.downloader

In [52]:
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-100')
VOCAB_SIZE = len(glove_vectors.vectors)
EMBEDDING_DIM = 100

This function will embed the dataset into sequences of 100-dimension vectors.

In [53]:
# pad dataset to a maximum review length in words
MAX_LEN = 50

def get_glove_seq(review, max_len):
  seq = np.zeros((max_len, 100))
  for i, word in enumerate(review):
    if i < max_len and word in glove_vectors:
      seq[i] = glove_vectors[word]
  return seq

In [54]:
news_data_train = load_dataset("ag_news", split="train").shuffle()
news_data_test = load_dataset("ag_news", split="test").shuffle()
NEWS_TRAIN = pd.DataFrame(news_data_train)[:5000]
NEWS_TEST = pd.DataFrame(news_data_test)[:5000]
NUM_LABELS = 4

In [55]:
NEWS_TEST.head()

,text,label
0,US rates seen headed up despite soft-spot worr...,2
1,"Nortel Cuts Fewer Jobs, Exits Real Estate OTT...",2
2,Movie Studios to Sue Illegal Film-File Traders...,3
3,UPDATE 2-Manchester United calls off bid talks...,1
4,Wholesale prices post biggest gain since 1990 ...,2


Train/Test Sets using GloVe embeddings.

In [56]:
# Vectorize text reviews to numbers
X_NEWS_TRAIN = np.zeros((len(NEWS_TRAIN), MAX_LEN, 100))
X_NEWS_TEST = np.zeros((len(NEWS_TEST), MAX_LEN, 100))

for i, row in NEWS_TRAIN.iterrows():
  X_NEWS_TRAIN[i] = get_glove_seq(tokenize_and_clean(row['text'], stem_and_remove_stop_words=False), MAX_LEN)

for i, row in NEWS_TEST.iterrows():
  X_NEWS_TEST[i] = get_glove_seq(tokenize_and_clean(row['text'], stem_and_remove_stop_words=False), MAX_LEN)

Y_NEWS_TRAIN = np.array(NEWS_TRAIN['label'])
Y_NEWS_TEST = np.array(NEWS_TEST['label'])
NUM_LABELS = 4

In [58]:
# Defining neural network structure
class MultinomialBoWClassifier(nn.Module):  # inheriting from nn.Module!
  def __init__(self, max_word_len, embedding_dim, num_labels):
    super(MultinomialBoWClassifier, self).__init__()
    self.max_word_len = max_word_len
    self.embedding_dim = embedding_dim
    self.num_labels = num_labels
    ### BEGIN SOLUTION
    # ok so should be somewhat similar to above
    # first we gotta flatten embeddings
    self.flat_layer = nn.Flatten()
    # first layer
    # made up 256, not sure here, might have to revisit
    self.lin1 = nn.Linear(max_word_len * embedding_dim, 256)
    # sigmoid for activation per usual
    self.sigmoid = nn.Sigmoid()
    # notes said try dropout before the way out so well give it a shot, 0.35 is a guess
    self.dropout = nn.Dropout(0.35)
    # out, 256 to connect everything with above
    self.lin2 = nn.Linear(256, num_labels)
    ### END SOLUTION

  def forward(self, x):
    out = None
    ### BEGIN SOLUTION
    # just connecting all the above really
    x = self.flat_layer(x)
    # with activation
    x = self.sigmoid(self.lin1(x))
    x = self.dropout(x)
    out = self.lin2(x)

    ### END SOLUTION
    return out

In [59]:
multibow_model = MultinomialBoWClassifier(max_word_len=MAX_LEN, embedding_dim=EMBEDDING_DIM, num_labels=NUM_LABELS)

In [60]:
# Train the model
def train(model, x_train_data, y_train_data, epochs, batch_size, lr, weight_decay):
  print('Training Started!')
  optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
  criterion = nn.CrossEntropyLoss()
  n_iter = len(x_train_data) // batch_size
  print(n_iter, 'batches per epoch')

  for epoch in range(epochs):
    num_correct = 0
    total_loss = 0.0
    model.train()

    for i in range(n_iter):
      x, y = get_batch(i, batch_size, x_train_data, y_train_data)
      x = x
      y = y.long()

      y_hat = model(x)
      loss = criterion(y_hat, y)
      total_loss += loss
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if (epoch+1)%10 == 0 and (i+1) == n_iter:
        print('epoch:', epoch+1,',loss =',loss.item(), ', training accuracy =',(y_hat.argmax(dim=1)==y).float().mean().item())

In [ ]:
# It's ok to modify this cell.
BATCH_SIZE = 50
N_EPOCHS = 100
LEARNING_RATE = 2e-3
WEIGHT_DECAY = 1e-2

In [ ]:
try:
    train(multibow_model, X_NEWS_TRAIN, Y_NEWS_TRAIN, N_EPOCHS, BATCH_SIZE, lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
except:
    print("Training failed. Please check your code.")

# Multinomial Regression - Test (40 Points)

In [ ]:
# student check - accuracies >= 80% will receive full credit (no credit for less than 80%)
ag.test_model_accuracy_mr(X_NEWS_TEST, Y_NEWS_TEST, multibow_model)

# Grading

Please submit this .ipynb file to Gradescope for grading.

## Final Grade

In [ ]:
# student check
ag.FINAL_GRADE()

## Notebook Runtime

In [ ]:
# end time - notebook execution
end_nb = time.time()
# print notebook execution time in minutes
print("Notebook execution time in minutes =", (end_nb - start_nb)/60)
# warn student if notebook execution time is greater than 30 minutes
if (end_nb - start_nb)/60 > 30:
  print("WARNING: Notebook execution time is greater than 30 minutes. Your submission may not complete auto-grading on Gradescope. Please optimize your code to reduce the notebook execution time.")